# Prefer Raising Exceptions to Returning `None`

In [1]:
# In some cases, returning None seems to make sense
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

In [2]:
x, y = 1, 0
result = careful_divide(x, y)
if result is None:
    print('Invalid inputs')

Invalid inputs


What happens with the `careful_divide` function when the numerator is zero? If the denominator is not zero, the function returns zero. The problem is that a zero return value can cause issues when you evaluate the result in a condition like an `if` statement. You might accidentally look for any `False`-equivalent value to indicate errors instead of only looking for None. 

In [3]:
# This code illustrates the problem described above
x, y = 0, 5
result = careful_divide(x, y)
if not result:
    print('Invalid inputs') # This runs! But it shouldn't

Invalid inputs


The misinterpretation of a `False`-equivalent return value is a common mistake in Python code when `None` has special meaning. This is why returning `None` from a function like `careful_divide` is error prone. There are two ways to reproduce the chance of such errors:

In [4]:
# The first way is to split the result value into a two-tuple. The first part of the tuple indicates that the
# operation was a success or a failure. The second part is the actual result that was computed
def careful_divide(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None

In [5]:
# Callers of the above function have to unpack the tuple. That forces them to consider the status part of the 
# tuple instead of just looking at the result division
success, result = careful_divide(x, y)
if not success:
    print('Invalid inputs')

In [6]:
# The problem is that callers can easily ignore the first part of the tuple. The resulting code doesn't look
# wrong at first glance, but this can be just as error prone as returning None
_, result = careful_divide(x, y)
if not result:
    print('Invalid inputs')

Invalid inputs


In [7]:
# The second, better way to reduce these errors is to never return None for special cases. Instead, raise an
# Exception up to the caller and have the caller deal with it
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

In [8]:
# The caller no longer requires a condition on the return value of the function. Instead, it can assume that the
# return value is always valid and use the results immediately in the else block after try
x, y = 5, 2
try:
    result = careful_divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.f' % result)

Result is 2


In [9]:
# This approach can be extended using to code using type annotations and docstrings to document exception-raising
# behavior
def careful_divide(a: float, b: float) -> float:
    '''Divided a by b
    
    Raises:
        ValueError: When the inputs cannot be divided
    '''
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')